In [1]:
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import cv2

In [3]:
import numpy as np
from collections import deque

In [4]:
x=160
y=25
h=370
w=345
clr = (0, 255, 0)
brd = 2

y_0 = y+brd
y_1 = y+h-brd-1
x_0 = x+brd
x_1 = x+w-brd-1

IMG_ROWS = 48
IMG_COLS = 48

In [5]:
classes_dict = {
    0: "book",
    1: "film",
    2: "show",
    3: "geo",
    4: "phrase",
    5: "series",
    6: "one",
    7: "two",
    8: "three",
    9: "four",
    10: "five",
    11: "six",
    12: "seven",
    13: "eight",
    14: "nine",
    15: "ten"
}

In [6]:
img_count = 0

In [7]:
THE_CHOSEN_MODEL = "models/Charades-CNN-48x3-ThreeDropout-Batch-32-l1-0x001"

In [8]:
saved_model = load_model(THE_CHOSEN_MODEL + '.model')

In [13]:
def preprocess(img):
    # izecanje regiona od znacaja
    crop_img = img[y_0:y_1, x_0:x_1]
    # otkljanjanje suma -> gausovsko zamucivanje
    blur_img = cv2.GaussianBlur(crop_img, (5,5), 0)
    # dodavanje maske u tonu koze -> hsv (dobar za izdvajanje objekata odredjene boje sa slike)
    # konverzija u hsv prstor boja
    hsv_img = cv2.cvtColor(blur_img, cv2.COLOR_BGR2HSV)
    # odredjivanje donje i gornje granice boje pod koje spada boja koze
    lower_hsv = (0, 7, 55)
    upper_hsv = (15, 190, 255)
    # tresholding pomocu inRange
    hsv_mask = cv2.inRange(hsv_img, lower_hsv, upper_hsv)
    # odstranjujemo sum
    hsv_blur_mask = cv2.medianBlur(hsv_mask, 3)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    hsv_ero = cv2.erode(hsv_blur_mask, kernel, iterations=1)
    hsv_open = cv2.dilate(hsv_ero, kernel, iterations=4)
    # konture
    contours, hierarchy = cv2.findContours(hsv_open, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    approved_contours = []
    for contour in contours:
        center, size, angle = cv2.minAreaRect(contour)
        width, height = size
        if width > 60 and height > 50:
            approved_contours.append(contour)
    # obradjena slika
    height, width, channels = crop_img.shape
    black = np.zeros((height, width))

    cv2.fillPoly(black, approved_contours, color=(255,255,255))
    # povratna vrednost -> ulaz za mrezu
    cv2.imshow("preprocesed", black)
    #cv2.imwrite("pre/pre_"+str(img_count)+".png", black)
    return black

In [10]:
def charades(frame):
    p_frame = preprocess(frame)
    image = p_frame / 255.0
    image = cv2.resize(image, (IMG_ROWS, IMG_COLS))
    image = image.flatten()
    img_arr = np.array(image).reshape(-1, IMG_ROWS, IMG_COLS, 1)
    
    '''
    prediction_proba = saved_model.predict_proba(img_arr)
    best_pred_value = np.amax(prediction_proba) * 100
    print("value", best_pred_value)
    best_pred = np.argmax(prediction_proba)
    print("best prediction", classes_dict[best_pred])
    '''
    
    prediction = saved_model.predict(img_arr)
    #prediction = saved_model.predict_classes(img_arr)
    #pred_class = classes_dict[prediction[0]]
    return prediction

In [19]:
# load images from camera

cam = cv2.VideoCapture(0)

cv2.namedWindow("Play charades!")

N_frames = 32

img_count = 0
charades_class = ""

current_action_deque = deque(maxlen=N_frames)

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break

    #cv2.rectangle(frame, (x,y), (x+w,y+h), clr, brd)

    frame = cv2.flip(frame, 1)
    
    cv2.putText(frame,  
                "Let's play some charades",  
                (120, 50),  
                cv2.FONT_HERSHEY_SIMPLEX, 1,  
                (0, 255, 255), 
                1, 
                cv2.LINE_AA)
    
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    
    '''
    # hvatanje frejma pomocu space tastera
    elif k%256 == 32:
        # SPACE pressed
        img_count += 1
        charades_class = charades(frame)
        print(charades_class)
    '''
    
    img_count += 1
    charades_preds = charades(frame)
    # dodavanje najnovije predikcije u deque
    current_action_deque.append(charades_preds[0])

    # odredjivanje prosecne predikcije za prethodnih N frejmova (0.5 sekundi otprilike)
    mean_pred = np.array(current_action_deque).mean(axis=0)
    mean_pred_idx = np.argmax(mean_pred)
    charades_class = classes_dict[mean_pred_idx]
    print(charades_class)
    
    cv2.putText(frame,  
            "Found: " + charades_class,  
            (50, 400),  
            cv2.FONT_HERSHEY_SIMPLEX, 1,  
            (0, 255, 255), 
            1, 
            cv2.LINE_AA)
    
    cv2.imshow("Play charades!", frame)

cam.release()

cv2.destroyAllWindows()

six
six
six
six
six
six
six
six
six
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
seven
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
eight
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
five
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase
phrase

In [18]:
# check fps
import numpy as np 
import cv2 
import time 
  
  
# creating the videocapture object 
# and reading from the input file 
# Change it to 0 if reading from webcam 
  
cap = cv2.VideoCapture(0) 
  
# used to record the time when we processed last frame 
prev_frame_time = 0
  
# used to record the time at which we processed current frame 
new_frame_time = 0
  
# Reading the video file until finished 
while(cap.isOpened()): 
  
    # Capture frame-by-frame 
  
    ret, frame = cap.read() 
  
    # if video finished or no Video Input 
    if not ret: 
        break
  
    # Our operations on the frame come here 
    gray = frame 
  
    # resizing the frame size according to our need 
    gray = cv2.resize(gray, (500, 300)) 
  
    # font which we will be using to display FPS 
    font = cv2.FONT_HERSHEY_SIMPLEX 
    # time when we finish processing for this frame 
    new_frame_time = time.time() 
  
    # Calculating the fps 
  
    # fps will be number of frame processed in given time frame 
    # since their will be most of time error of 0.001 second 
    # we will be subtracting it to get more accurate result 
    fps = 1/(new_frame_time-prev_frame_time) 
    prev_frame_time = new_frame_time 
  
    # converting the fps into integer 
    fps = int(fps) 
  
    # converting the fps to string so that we can display it on frame 
    # by using putText function 
    fps = str(fps) 
    print(fps)
  
    # puting the FPS count on the frame 
    #cv2.putText(gray, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA) 
  
    # displaying the frame with fps 
    cv2.imshow('frame', gray) 
  
    # press 'Q' if you want to exit 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
# When everything done, release the capture 
cap.release() 
# Destroy the all windows now 
cv2.destroyAllWindows() 

0
18
92
28
64
24
26
31
31
44
18
31
27
30
32
32
31
47
32
21
32
20
34
31
27
31
31
32
32
45
22
31
45
21
24
31
32
26
33
32
32
40
31
23
31
26
23
31
30
34
44
22
31
33
29
31
23
32
26
32
37
31
37
32
32
22
31
52
17
32
32
26
31
36
38
32
32
26
25
31
22
28
56
21
31
39
36
32
32
26
26
55
16
31
32
45
21
40
36
31
32
26
26
20
53
32
32
29
32
21
30
31
31
36
31
18
56
32
21
53
22
30
38
32
32
28
32
18
49
32
20
34
31
30
32
31
30
32
31
21
30
32
30
31
44
22
34
31
31
30
21
32
52
31
32
28
31
22
36
32
31
29
18
33
56
32
21
48
32
22
31
31
33
32
21
31
25
33
52
28
31
32
22
36
31
28
21
28
57
21
32
43
32
23
38
31
31
18
31
28
31
32
52
27
32
23
31
31
31
21
28
33
53
32
32
26
24
43
27
32
31
18
29
32
52
32
32
26
